<a href="https://colab.research.google.com/github/yonatanrtt/NER/blob/main/Dataset_on_roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers --quiet

     |████████████████████████████████| 365 kB 31.0 MB/s 
     |████████████████████████████████| 4.7 MB 43.8 MB/s 
     |████████████████████████████████| 212 kB 49.6 MB/s 
     |████████████████████████████████| 120 kB 47.5 MB/s 
     |████████████████████████████████| 115 kB 61.4 MB/s 
     |████████████████████████████████| 127 kB 60.7 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 


In [2]:
import pandas as pd

data = pd.read_csv('./demo_bot_intents.csv')

In [3]:
from datasets import Dataset

def conv_to_ds(_data, _is_train):
  filtered_df = _data[_data['is_train']==int(_is_train)][['text', 'lables']]
  filtered_df = filtered_df.rename(columns={"lables": "labels"})
  return Dataset.from_pandas(filtered_df)

train_dataset  = conv_to_ds(data, _is_train=True)
test_dataset = conv_to_ds(data, _is_train=False)

In [4]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(
        model_name,
        num_labels=5
        )
model = AutoModelForSequenceClassification.from_config(config)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=50, truncation=True)


train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./test_trainer", evaluation_strategy="epoch", num_train_epochs=30,learning_rate=5e-06)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 40
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 150


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.668939,0.200000
2,No log,1.589519,0.300000
3,No log,1.585955,0.325000
4,No log,1.565097,0.325000
5,No log,1.552176,0.400000
6,No log,1.536664,0.450000
7,No log,1.528224,0.350000
8,No log,1.515001,0.425000
9,No log,1.494357,0.450000
10,No log,1.470384,0.450000


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forwa

TrainOutput(global_step=150, training_loss=1.2842965698242188, metrics={'train_runtime': 828.3284, 'train_samples_per_second': 1.449, 'train_steps_per_second': 0.181, 'total_flos': 30834157320000.0, 'train_loss': 1.2842965698242188, 'epoch': 30.0})

In [9]:
trainer.save_model("./trained_model")

Saving model checkpoint to ./trained_model
Configuration saved in ./trained_model/config.json
Model weights saved in ./trained_model/pytorch_model.bin


In [10]:
!zip trained_model.zip trained_model

  adding: trained_model/ (stored 0%)


In [11]:
!zip trained_model.zip trained_model

updating: trained_model/ (stored 0%)
